PS：因为无法找到作业来源，所以只能按着ppt自己从头实现。如果有热心的小伙伴找到了并且想要更新这个项目，可以联系我的邮箱。

# 问题描述

希尔密码是通过矩阵乘法来进行加密的方法，我们需要解决三个问题：

1. 给出key和明文，来求得密文，主要步骤如下：
    * 将明文转为矩阵形式
    * 求解密文矩阵
    * 将密文从矩阵形式转为文字形式
2. 给出密文和key，来求得明文，主要步骤如下：
    * 将密文转化为矩阵形式
    * 求解key的模运算下的逆矩阵
    * 求解明文矩阵
    * 将明文从矩阵形式转化为文字形式
3. 给出明文和密文，来求得key，主要步骤如下：
    * 将明文和密文转化为矩阵形式
    * 求解明文的模运算下的逆矩阵
    * 求解key矩阵

那么不难看出，我们需要实现的主要是三个函数：

1. 将文本转化为矩阵
2. 将矩阵转化为文本
3. 求矩阵的模逆阵



In [1]:
# 建立字符集合对照表
table = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'J':9, 
'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19, 
'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, '_':26, '.':27, ',':28, '?':29,
'!':30}

inv_table = {v: k for k, v in table.items()}


In [2]:
import numpy as np

# 将文本转化为矩阵
def text2matrix(text):
    '''
    input
    text: 文本
    output
    matrix: 文本对应的矩阵
    '''
    # if n=3
    l = len(text)
    col = l // 3
    mod = l % 3
    m = np.zeros((3, col),'int')
    for i in range(3):
        for j in range(col):
            m[i,j] = table[text[i+j*3]]
    if mod>0:
        v = np.zeros((3,), 'int')
        for i in range(mod):
            v[i] = table[text[col*3+i]]
        # 补上末尾重复元素
        for i in range(mod, 3):
            v[i] = table[text[col*3+mod-1]]
        m = np.column_stack((m,v))
    return m
    
# 测试
# 输入 THIS_IS_AN_APPLE.
# 输出
'''
[
    19 18 18 13 15 4
    7  26 26 26 15 27
    8  8  0  0  11 27
]
'''
    
text2matrix("THIS_IS_AN_APPLE.")



array([[19, 18, 18, 13, 15,  4],
       [ 7, 26, 26, 26, 15, 27],
       [ 8,  8,  0,  0, 11, 27]])

In [3]:
# 将矩阵转化为文本
def matrix2text(m):
    '''
    input
    matrix: 文本对应的矩阵
    output
    text: 文本
    '''
    # if n=3
    res = ""
    for j in range(m.shape[1]):
        for i in range(m.shape[0]):
            res += inv_table[m[i,j]]
    # 去除补上的重复元素
    while res[-1] == res[-2]:
        res = res[:-1]
    
    return res


# 测试
# 输入
'''
[
    19 18 18 13 15 4
    7  26 26 26 15 27
    8  8  0  0  11 27
]
'''
# 输出 THIS_IS_AN_APPLE.

matrix2text(np.array([[19, 18, 18, 13, 15, 4], [7,26,26,26,15,27], [8,8,0,0,11,27]]))


'THIS_IS_AN_APPLE.'

In [4]:
#求解矩阵的模逆

def mod_inverse(m, S):
    '''
    input
    S: 字母表的基数
    matrix: 待求逆的矩阵
    output
    inverse: 矩阵的模逆
    '''
    # if n=3 因为A·A*(伴随矩阵)=|A|E 
    # 所以inv(a)(mod |S|)= ((1/|A|)(mod |S|) * A)(mod |S|)
    d = round(np.linalg.det(m))
    

    # 利用简单试错法求解 1/d(mod|S|)，其中|S|是字符集的个数，为31
    inv_d = 0
    for i in range(1, S):
        if d*i%S == 1:
            inv_d = i
            break

    adj = np.zeros(m.shape, 'int')
    # 求伴随矩阵
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            row = list(range(0, i)) + list(range(i+1, m.shape[0]))
            col = list(range(0, j)) + list(range(j+1, m.shape[1]))
            adj[j, i] = ((-1)**(i+j)) * round(np.linalg.det(m[row][:,col]))
    
    return inv_d * adj % S

# 测试
# 输入
'''
[
    4, 9, -2
    3, 5, 7
    1, -6, 11
]
'''
# 输出
'''
[
    16, 0, 24
    28, 4, 21
    27, 18, 12
]
'''

mod_inverse(np.array([[4, 9, -2], [3, 5, 7], [1, -6, 11]]), 29)


array([[16,  0, 24],
       [28,  4, 21],
       [27, 18, 12]], dtype=int32)

到这里就有可能就有小伙伴会问了，如果矩阵不可逆咋办呢？那是不行滴，key必须设计成可逆的，而且|S|最好是质数，感兴趣的小伙伴可以看看这篇[文章](https://ccjou.wordpress.com/2013/09/10/%E5%B8%8C%E7%88%BE%E5%AF%86%E7%A2%BC/?fbclid=IwAR175SK34eqCXJfhOsDnlk_0cEQ4bLSDG1BSSsd36JQSMaq436LxlpJoIok)，里面有解释“如何利用模运算判定矩阵可逆性”和“有限体和模运算”

不想深入了解的小伙伴也不用担心，老师给的数据都是可逆的，不用担心~

In [5]:
# 测试
# 明文：IS_THAT_W
# 密文：C!QER,YNR
# key
'''
[
    25, 8, 25,
    9,  9, 16,
    28, 21,18
]
'''

plain = "IS_THAT_W"
cipher = "C!QER,YNR"
key = np.array([[25, 8, 25],
                [9,  9, 16],
                [28, 21, 18]])


In [6]:
# problem 1 已知key和明文文，求密文
print("cipher", "is", matrix2text(np.matmul(key, text2matrix(plain))%31))

cipher is C!QER,YNR


In [7]:
# problem 2 已知key和密文，求明文

print("plain", "is", matrix2text(np.matmul(mod_inverse(key, 31), text2matrix(cipher)) % 31))


plain is IS_THAT_W


In [8]:
# problem 3 已知明文和密文，求key
np.matmul(text2matrix(cipher), mod_inverse(text2matrix(plain), 31)) % 31


array([[25,  8, 25],
       [ 9,  9, 16],
       [28, 21, 18]], dtype=int32)